In [1]:
from functions.Pre_processing_Casos_Graves import *
from functions.Random_forest import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection._search import GridSearchCV
from sklearn.model_selection._split import StratifiedKFold
from sklearn import svm

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, SelectFdr

import optuna
from sklearn.metrics import f1_score, precision_score, recall_score

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def objective(trial):
        classifier_name = trial.suggest_categorical("classifier", ["RandomForest"])

        bootstrap = trial.suggest_categorical("bootstrap", [True, False])
        #max_depth = trial.suggest_int("max_depth", 10, 100)
        criterion = trial.suggest_categorical("criterion", ['gini','entropy'])
        max_features = trial.suggest_categorical("max_features", ['auto', 'sqrt','log2'])
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        n_estimators = trial.suggest_int("n_estimators", 100, 2000)

        classifier_obj = RandomForestClassifier(
            bootstrap=bootstrap, max_depth=None,
            criterion=criterion, max_features=max_features,
            min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
            n_estimators=n_estimators, random_state=42, n_jobs=-1)

        score = cross_val_score(classifier_obj, X_train, y_train, n_jobs=-1, cv=5,scoring='f1')
        f1 = score.mean()
        return f1

In [2]:
# casos_graves = Pre_Processing_Casos_Graves()
# casos_graves.run(columns_symptoms,columns_to_drop,columns_dieases,path)

In [3]:
df = pd.read_csv(r'C:\Users\gabri\Desktop\Git - Tcc\Estudo_dados_de_covid\Data\Casos Graves\df_casos_graves')

In [4]:
random_forest = Random_forest(df)

In [5]:
random_forest.run_graves()

In [13]:
df = random_forest.df

In [20]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death', 'idade', 'sexo_Feminino', 'sexo_Ignorado','sexo_Masculino']

In [21]:
df = df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Ignorado,sexo_Masculino
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0.460870,0,0,1
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.747826,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.573913,0,0,1
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.591304,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.669565,0,0,1
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.643478,0,0,1
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.643478,0,0,1
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.634783,1,0,0


In [22]:
X = df.drop("death",axis=1)
y = df['death']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-08-31 16:25:02,361] A new study created in memory with name: no-name-19e53208-6e5e-44f0-b027-d85734ef13e6
[I 2022-08-31 16:25:52,969] Trial 0 finished with value: 0.3927469081789179 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 1780}. Best is trial 0 with value: 0.3927469081789179.
[I 2022-08-31 16:25:59,336] Trial 1 finished with value: 0.36346317339870154 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 279}. Best is trial 0 with value: 0.3927469081789179.
[I 2022-08-31 16:26:22,264] Trial 2 finished with value: 0.3414629832195003 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 1257}. Best is

FrozenTrial(number=45, values=[0.404500594479674], datetime_start=datetime.datetime(2022, 8, 31, 16, 49, 40, 576669), datetime_complete=datetime.datetime(2022, 8, 31, 16, 50, 19, 649484), params={'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1842}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=45, state=TrialState.COMPLETE, value=None)


In [25]:
best_params={'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1842}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-p

Média de fit_time: 31.610836815834045
Média de score_time: 1.1915322065353393
Média de test_f1: 0.3976067102650175
Média de test_precision: 0.48969581252218514
Média de test_recall: 0.3348300872548709


In [26]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Precisão: 0.4676079734219269
Revocação: 0.3112216694306247
F1-score: 0.37371390640557584


## Aplicando o modelo para intervalo de tempo de 3 meses

intervalos:

In [4]:
periodo_1 = df[(df['data_notificacao'] > '2020-02-26') & (df['data_notificacao'] < '2020-05-31')]
periodo_2 = df[(df['data_notificacao'] > '2020-06-01') & (df['data_notificacao'] < '2020-08-31')]
periodo_3 = df[(df['data_notificacao'] > '2020-09-01') & (df['data_notificacao'] < '2020-11-30')]
periodo_4 = df[(df['data_notificacao'] > '2020-12-01') & (df['data_notificacao'] < '2021-02-28')]
periodo_5 = df[(df['data_notificacao'] > '2021-03-01') & (df['data_notificacao'] < '2021-05-31')]
periodo_6 = df[(df['data_notificacao'] > '2021-06-01') & (df['data_notificacao'] < '2021-08-31')]
periodo_7 = df[(df['data_notificacao'] > '2021-09-01') & (df['data_notificacao'] < '2021-11-30')]
periodo_8 = df[(df['data_notificacao'] > '2021-12-01') & (df['data_notificacao'] < '2022-02-28')]
periodo_9 = df[(df['data_notificacao'] > '2022-03-01') & (df['data_notificacao'] < '2025-05-31')]
periodo_10 = df[(df['data_notificacao'] > '2022-06-01') & (df['data_notificacao'] < '2022-06-13')]

Lista de features atulizadas no modelo

In [15]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death', 'idade', 'sexo_Feminino','sexo_Masculino']


### Aplicando para o periodo de 26/02/22 - 31/05/20

In [7]:
random_forest = Random_forest(periodo_1)
random_forest.run_graves()
df = random_forest.df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Ignorado,sexo_Masculino
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0.486239,0,0,1
24,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.633028,0,0,1
25,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.642202,0,0,1
26,1,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0.596330,0,0,1
27,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.467890,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36271,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.532110,1,0,0
36272,1,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0.871560,1,0,0
36274,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0.614679,0,0,1
36275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.623853,0,0,1


In [8]:
X = df.drop("death",axis=1)
y = df['death']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-09-02 15:08:40,128] A new study created in memory with name: no-name-64eb69fb-8533-48cc-8628-491acb69073e
[I 2022-09-02 15:08:55,740] Trial 0 finished with value: 0.517577535907628 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1856}. Best is trial 0 with value: 0.517577535907628.
[I 2022-09-02 15:09:03,170] Trial 1 finished with value: 0.5330536240469581 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 513}. Best is trial 1 with value: 0.5330536240469581.
[I 2022-09-02 15:09:14,661] Trial 2 finished with value: 0.4911341926529813 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1885}. Best is trial 1 w

FrozenTrial(number=42, values=[0.5444618431366075], datetime_start=datetime.datetime(2022, 9, 2, 15, 13, 13, 402196), datetime_complete=datetime.datetime(2022, 9, 2, 15, 13, 21, 161202), params={'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 1100}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=42, state=TrialState.COMPLETE, value=None)


In [11]:
best_params={'bootstrap': True, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 1100}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-p

In [12]:
for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 5.622239255905152
Média de score_time: 0.2656330823898315
Média de test_f1: 0.5415081645042694
Média de test_precision: 0.6163804340767587
Média de test_recall: 0.48561027486911


In [13]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Precisão: 0.6014492753623188
Revocação: 0.535483870967742
F1-score: 0.5665529010238908


### Aplicando para o periodo de 01/06/20 - 31/08/20

In [16]:
random_forest = Random_forest(periodo_2)
random_forest.run_graves()
df = random_forest.df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.803738,1,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.635514,0,1
5,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.869159,0,1
6,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.626168,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36251,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.644860,1,0
36252,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.654206,1,0
36261,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.747664,0,1
36270,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.766355,1,0


In [17]:
X = df.drop("death",axis=1)
y = df['death']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-09-02 15:20:39,465] A new study created in memory with name: no-name-5906a1d6-c179-46c3-8d2b-eb021cce6c54
[I 2022-09-02 15:20:42,323] Trial 0 finished with value: 0.20778578571118028 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 324}. Best is trial 0 with value: 0.20778578571118028.
[I 2022-09-02 15:20:51,272] Trial 1 finished with value: 0.3174499094641753 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1871}. Best is trial 1 with value: 0.3174499094641753.
[I 2022-09-02 15:20:53,347] Trial 2 finished with value: 0.12871610280290358 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 715}. Best is t

FrozenTrial(number=38, values=[0.35635803085892853], datetime_start=datetime.datetime(2022, 9, 2, 15, 23, 19, 19160), datetime_complete=datetime.datetime(2022, 9, 2, 15, 23, 23, 898793), params={'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 862}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=38, state=TrialState.COMPLETE, value=None)


In [20]:
best_params={'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 862}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

In [21]:
for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 3.4026634454727174
Média de score_time: 0.12488048076629639
Média de test_f1: 0.33701557902061663
Média de test_precision: 0.3833716921659235
Média de test_recall: 0.302037037037037


In [22]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

Precisão: 0.4266666666666667
Revocação: 0.30917874396135264
F1-score: 0.35854341736694684


### Aplicando para o periodo de 01/09/20 - 30/11/20

In [23]:
random_forest = Random_forest(periodo_3)
random_forest.run_graves()
df = random_forest.df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
431,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0.807692,0,1
432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.182692,0,1
433,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.875000,1,0
434,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.548077,0,1
436,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.807692,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36168,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0.576923,1,0
36169,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.586538,0,1
36235,1,0,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0.605769,0,1
36236,1,0,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0.519231,0,1


In [24]:
X = df.drop("death",axis=1)
y = df['death']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-09-02 15:29:01,366] A new study created in memory with name: no-name-366bbd02-76a6-4bf4-be52-c0dd7711d1aa
[I 2022-09-02 15:29:05,312] Trial 0 finished with value: 0.3349941493691494 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1664}. Best is trial 0 with value: 0.3349941493691494.
[I 2022-09-02 15:29:07,733] Trial 1 finished with value: 0.33448953555728134 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 642}. Best is trial 0 with value: 0.3349941493691494.
[I 2022-09-02 15:29:08,468] Trial 2 finished with value: 0.33556125299067713 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 7, 'n_estimators': 330}. Best is t

FrozenTrial(number=30, values=[0.3993281895678155], datetime_start=datetime.datetime(2022, 9, 2, 15, 30, 2, 772877), datetime_complete=datetime.datetime(2022, 9, 2, 15, 30, 4, 22606), params={'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 463}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=30, state=TrialState.COMPLETE, value=None)


In [27]:
best_params={'bootstrap': False, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 463}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

In [28]:
for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 0.8653348922729492
Média de score_time: 0.05450558662414551
Média de test_f1: 0.3933860082888324
Média de test_precision: 0.433928156608214
Média de test_recall: 0.36188405797101453


In [29]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

Precisão: 0.5172413793103449
Revocação: 0.3629032258064516
F1-score: 0.4265402843601896


### Aplicando para o periodo de 01/12/20 - 28/02/21

In [30]:
random_forest = Random_forest(periodo_4)
random_forest.run_graves()
df = random_forest.df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
937,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.517544,1,0
938,1,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0.350877,0,1
939,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.771930,1,0
970,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.026316,1,0
979,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.456140,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36245,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.666667,1,0
36246,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.570175,1,0
36247,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.429825,0,1
36351,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0.798246,1,0


In [31]:
X = df.drop("death",axis=1)
y = df['death']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-09-02 15:34:09,197] A new study created in memory with name: no-name-2df5180e-1946-49f4-ba50-b43fc99f03aa
[I 2022-09-02 15:34:09,626] Trial 0 finished with value: 0.5002048036633339 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 183}. Best is trial 0 with value: 0.5002048036633339.
[I 2022-09-02 15:34:11,834] Trial 1 finished with value: 0.5103462199724871 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 884}. Best is trial 1 with value: 0.5103462199724871.
[I 2022-09-02 15:34:13,062] Trial 2 finished with value: 0.4818679175500476 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 482}. Best is trial 1 wi

FrozenTrial(number=45, values=[0.520552940894443], datetime_start=datetime.datetime(2022, 9, 2, 15, 35, 34, 301646), datetime_complete=datetime.datetime(2022, 9, 2, 15, 35, 36, 408652), params={'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 910}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=45, state=TrialState.COMPLETE, value=None)


In [34]:
best_params={'bootstrap': False, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 910}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

In [35]:
for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 1.5777292490005492
Média de score_time: 0.09266033172607421
Média de test_f1: 0.5244994008637721
Média de test_precision: 0.6363506407537943
Média de test_recall: 0.4466866466866467


In [36]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

Precisão: 0.5539568345323741
Revocação: 0.4074074074074074
F1-score: 0.46951219512195125


### Aplicando para o periodo de 01/03/21 - 31/05/21

In [37]:
random_forest = Random_forest(periodo_5)
random_forest.run_graves()
df = random_forest.df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
952,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.429825,0,1
953,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.315789,1,0
954,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.368421,1,0
955,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.394737,0,1
956,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.377193,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36332,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.333333,1,0
36333,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.631579,1,0
36334,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0.438596,1,0
36362,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.491228,1,0


In [38]:
X = df.drop("death",axis=1)
y = df['death']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-09-02 15:43:54,838] A new study created in memory with name: no-name-a1f7c760-be56-44c0-adbb-68ab62e6d558
[I 2022-09-02 15:43:58,473] Trial 0 finished with value: 0.473924182555647 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 412}. Best is trial 0 with value: 0.473924182555647.
[I 2022-09-02 15:44:01,869] Trial 1 finished with value: 0.4763349536063606 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 433}. Best is trial 1 with value: 0.4763349536063606.
[I 2022-09-02 15:44:05,951] Trial 2 finished with value: 0.48864770938197866 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 1291}. Best is trial 

FrozenTrial(number=21, values=[0.493783463234581], datetime_start=datetime.datetime(2022, 9, 2, 15, 45, 20, 192518), datetime_complete=datetime.datetime(2022, 9, 2, 15, 45, 24, 80949), params={'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1183}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=21, state=TrialState.COMPLETE, value=None)


In [41]:
best_params={'bootstrap': True, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1183}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-p

In [42]:
for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 2.88069372177124
Média de score_time: 0.1485074758529663
Média de test_f1: 0.4801113760538387
Média de test_precision: 0.6245798083692568
Média de test_recall: 0.3931186868686869


In [43]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Precisão: 0.6739130434782609
Revocação: 0.41778975741239893
F1-score: 0.5158069883527454


Apartir da aplcação desses modelos nota-se que as métricas para o random forest não são boas